In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from itertools import product

# ============================================================================
# CONFIGURATION
# ============================================================================
FILE_PATH = r"C:\Users\abqua\OneDrive - QuantMatrix AI Solutions\hyou\phase-2\Data for model\Data_for_model.xlsx"
PRODUCT_TITLE = "interlocking diamond necklace"
TARGET_VARIABLE = "Net items sold"
STANDARDIZE = True
AUTO_TUNE_HYPERPARAMETERS = True

# Forward selection settings
MAX_VARIABLES = 3  # Total variables (3 mandatory + 7 selected)
MANDATORY_VARIABLES = ['Google trends', 'Product variant price', 'Category discount %']

# PREDICTION CONSTRAINTS
USE_LOG_TRANSFORM = False  # Transform to ensure positive predictions
CLIP_PREDICTIONS = True   # Clip to reasonable range
MIN_PREDICTION = 0        # Minimum allowed prediction
MAX_PREDICTION_MULTIPLIER = 3  # Max = mean * this value

# Fixed hyperparameters (if AUTO_TUNE = False)
Q_VALUE = 1e-4
R_VALUE = 1.0
INIT_COV = 1e3
# ============================================================================


class ConstrainedTVLinearKalman:
    def __init__(self, n_features: int, q: float = 1e-4, r: float = 1.0, init_cov: float = 1e3,
                 min_pred=0, max_pred=None, use_log=False):
        self.n = n_features
        self.q = q
        self.r = r
        self.init_cov = init_cov
        self.Q = np.eye(self.n) * self.q
        self.R = self.r
        self.I = np.eye(self.n)
        self.beta0 = np.zeros(self.n)
        self.P0 = np.eye(self.n) * self.init_cov
        
        # Constraints
        self.min_pred = min_pred
        self.max_pred = max_pred
        self.use_log = use_log

    def _step(self, x_t, y_t, beta_prev, P_prev, update=True):
        x_t = x_t.reshape(-1, 1)
        beta_pred = beta_prev.copy()
        P_pred = P_prev + self.Q
        y_pred_raw = (beta_pred @ x_t).item()
        
        # Apply constraints to prediction
        if self.use_log:
            # If using log transform, exp to get back to original scale
            y_pred = np.exp(y_pred_raw)
        else:
            y_pred = y_pred_raw
        
        # Clip prediction to valid range
        if self.min_pred is not None:
            y_pred = max(y_pred, self.min_pred)
        if self.max_pred is not None:
            y_pred = min(y_pred, self.max_pred)
        
        # Update step using the raw prediction (before clipping) for consistency
        if update and np.isfinite(y_t):
            resid = y_t - y_pred_raw
            S = (x_t.T @ P_pred @ x_t).item() + self.R
            K = (P_pred @ x_t) / S
            beta_upd = beta_pred + K.flatten() * resid
            temp = self.I - K @ x_t.T
            P_upd = temp @ P_pred @ temp.T + (K @ K.T) * self.R
        else:
            beta_upd = beta_pred.copy()
            P_upd = P_pred.copy()
        
        return beta_pred, P_pred, y_pred, beta_upd, P_upd

    def fit(self, X_train, y_train):
        T, p = X_train.shape
        betas_filt = np.zeros((T, p))
        y_pred = np.zeros(T)
        
        # Transform target if using log
        if self.use_log:
            # Add small constant to avoid log(0)
            y_train_transformed = np.log(y_train + 1)
        else:
            y_train_transformed = y_train
        
        beta = self.beta0.copy()
        P = self.P0.copy()
        
        for t in range(T):
            _, _, yhat, beta_upd, P_upd = self._step(
                X_train[t], y_train_transformed[t], beta, P, update=True
            )
            betas_filt[t] = beta_upd
            y_pred[t] = yhat
            beta = beta_upd
            P = P_upd
        
        return betas_filt, y_pred


def forward_selection(data, all_vars, y_var, max_vars=10, mandatory_vars=None, 
                     q=1e-4, r=1.0, init_cov=1e3, use_log=False):
    """Forward stepwise selection with constraints."""
    print("\n" + "="*80)
    print("🔍 FORWARD STEPWISE SELECTION (WITH CONSTRAINTS)")
    print("="*80)
    
    # Initialize with mandatory variables
    if mandatory_vars is None:
        mandatory_vars = []
    
    mandatory_vars_filtered = [v for v in mandatory_vars if v in data.columns and v in all_vars]
    
    if mandatory_vars_filtered:
        print(f"\n🔒 Mandatory variables (always included):")
        for i, var in enumerate(mandatory_vars_filtered, 1):
            print(f"   {i}. {var}")
        
        selected_vars = mandatory_vars_filtered.copy()
        remaining_vars = [v for v in all_vars if v not in mandatory_vars_filtered]
        num_to_select = max_vars - len(mandatory_vars_filtered)
        
        print(f"\n📊 Will select {num_to_select} more variables from {len(remaining_vars)} candidates...")
    else:
        selected_vars = []
        remaining_vars = all_vars.copy()
        num_to_select = max_vars
    
    selection_history = []
    y = data[y_var].values
    
    # Set constraints
    y_mean = np.mean(y)
    y_max = np.max(y)
    min_pred = 0
    max_pred = y_max * 3  # Allow up to 3x historical max
    
    for step in range(num_to_select):
        print(f"\n📍 Step {step + 1}: Testing {len(remaining_vars)} variables...")
        
        best_var = None
        best_r2 = -np.inf
        best_mae = np.inf
        
        for var in remaining_vars:
            test_vars = selected_vars + [var]
            
            try:
                X_test = data[test_vars].values
                scaler = StandardScaler()
                X_test_scaled = scaler.fit_transform(X_test)
                
                kf = ConstrainedTVLinearKalman(
                    n_features=len(test_vars), q=q, r=r, init_cov=init_cov,
                    min_pred=min_pred, max_pred=max_pred, use_log=use_log
                )
                _, y_pred = kf.fit(X_test_scaled, y)
                
                r2 = r2_score(y, y_pred)
                mae = mean_absolute_error(y, y_pred)
                
                if r2 > best_r2:
                    best_r2 = r2
                    best_mae = mae
                    best_var = var
            except:
                continue
        
        if best_var is None:
            print(f"   ⚠ No more variables to add")
            break
        
        selected_vars.append(best_var)
        remaining_vars.remove(best_var)
        
        selection_history.append({
            'Step': step + 1,
            'Variable': best_var,
            'R2': best_r2,
            'MAE': best_mae,
            'Num_Vars': len(selected_vars)
        })
        
        print(f"   ✓ Added: {best_var[:50]}")
        print(f"     R² = {best_r2:.4f}, MAE = {best_mae:.2f}")
    
    print(f"\n{'='*80}")
    print(f"📊 SELECTION SUMMARY")
    print(f"{'='*80}")
    
    if selection_history:
        history_df = pd.DataFrame(selection_history)
        print(history_df.to_string(index=False))
    else:
        history_df = pd.DataFrame()
    
    print(f"\n✅ Final selected variables ({len(selected_vars)}):")
    if mandatory_vars_filtered:
        print(f"\n   Mandatory variables ({len(mandatory_vars_filtered)}):")
        for i, var in enumerate(mandatory_vars_filtered, 1):
            print(f"   🔒 {i}. {var}")
        
        print(f"\n   Selected variables ({len(selected_vars) - len(mandatory_vars_filtered)}):")
        for i, var in enumerate(selected_vars):
            if var not in mandatory_vars_filtered:
                print(f"   ✓ {len(mandatory_vars_filtered) + 1}. {var}")
    
    if not history_df.empty:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=history_df['Step'],
            y=history_df['R2'],
            mode='lines+markers',
            line=dict(color='#2E86AB', width=3),
            marker=dict(size=10)
        ))
        fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Baseline")
        fig.update_layout(
            title="Forward Selection: R² Improvement",
            xaxis_title="Variables Added",
            yaxis_title="R²",
            height=500
        )
        fig.show()
    
    return selected_vars, history_df


def hyperparameter_tuning(X, y, n_features, use_log=False):
    """Grid search with constraints."""
    print(f"\n🔧 Hyperparameter tuning...")
    
    q_values = [1e-5, 1e-4, 1e-3, 1e-2]
    r_values = [0.1, 1.0, 10.0, 100.0]
    init_cov_values = [1e2, 1e3]
    
    best_r2 = -np.inf
    best_params = None
    
    y_max = np.max(y)
    
    for q, r, init_cov in product(q_values, r_values, init_cov_values):
        try:
            kf = ConstrainedTVLinearKalman(
                n_features=n_features, q=q, r=r, init_cov=init_cov,
                min_pred=0, max_pred=y_max*3, use_log=use_log
            )
            _, y_pred = kf.fit(X, y)
            r2 = r2_score(y, y_pred)
            
            if r2 > best_r2:
                best_r2 = r2
                best_params = (q, r, init_cov)
        except:
            continue
    
    print(f"   Best: Q={best_params[0]}, R={best_params[1]}, R²={best_r2:.4f}")
    return best_params


def calculate_metrics(y_true, y_pred):
    """Calculate metrics."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else np.nan
    
    return {'mae': mae, 'rmse': rmse, 'r2': r2, 'mape': mape}


def plot_results(y_true, y_pred, dates, betas, var_names, target_name):
    """Create plots."""
    
    # Plot 1: Predictions
    fig1 = make_subplots(rows=2, cols=1, row_heights=[0.6, 0.4],
                         subplot_titles=('Actual vs Predicted (CONSTRAINED)', 'Residuals'))
    
    fig1.add_trace(go.Scatter(x=dates, y=y_true, name='Actual', 
                              line=dict(color='#2E86AB', width=2)), row=1, col=1)
    fig1.add_trace(go.Scatter(x=dates, y=y_pred, name='Predicted (Constrained)', 
                              line=dict(color='#A23B72', width=2)), row=1, col=1)
    fig1.add_trace(go.Scatter(x=dates, y=[np.mean(y_true)]*len(dates), name='Mean', 
                              line=dict(color='orange', width=2, dash='dash')), row=1, col=1)
    
    # Add constraint lines
    fig1.add_hline(y=0, line_dash="dot", line_color="red", 
                   annotation_text="Min = 0", row=1, col=1)
    fig1.add_hline(y=np.max(y_true)*3, line_dash="dot", line_color="green", 
                   annotation_text="Max = 3x historical", row=1, col=1)
    
    residuals = y_true - y_pred
    fig1.add_trace(go.Scatter(x=dates, y=residuals, mode='markers', name='Residuals',
                              marker=dict(color=['red' if r<0 else 'green' for r in residuals])),
                   row=2, col=1)
    fig1.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)
    
    fig1.update_xaxes(title_text="Date", row=2, col=1)
    fig1.update_yaxes(title_text=target_name, row=1, col=1)
    fig1.update_yaxes(title_text="Residual", row=2, col=1)
    fig1.update_layout(height=900, title_text="Model Performance (Constrained)", showlegend=True)
    fig1.show()
    
    # Plot 2: Scatter with constraints
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=y_true, y=y_pred, mode='markers',
                              marker=dict(size=8, color=y_true, colorscale='Viridis')))
    
    min_val = 0
    max_val = max(y_true.max(), y_pred.max())
    fig2.add_trace(go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                              mode='lines', line=dict(color='red', dash='dash'),
                              name='Perfect'))
    
    # Show constraint boundaries
    fig2.add_hline(y=0, line_dash="dot", line_color="red", annotation_text="Min")
    fig2.add_hline(y=np.max(y_true)*3, line_dash="dot", line_color="green", 
                   annotation_text="Max")
    
    fig2.update_layout(
        title=f"Actual vs Predicted (R²={r2_score(y_true, y_pred):.4f})",
        xaxis_title=f"Actual {target_name}",
        yaxis_title=f"Predicted {target_name}",
        height=600, width=800
    )
    fig2.show()


def run_analysis():
    """Main analysis with constraints."""
    print("="*80)
    print("🔬 KALMAN FILTER - CONSTRAINED VERSION")
    print("="*80)
    
    # Load data
    print(f"\n📁 Loading data...")
    df = pd.read_excel(FILE_PATH)
    
    data = df[df['Product title'] == PRODUCT_TITLE].copy()
    print(f"✓ {len(data)} rows for '{PRODUCT_TITLE}'")
    
    # Get predictors
    all_impression_cols = [col for col in data.columns if 'impression' in col.lower()]
    mandatory_candidates = ['Product variant price', 'Category Discount', 'google_trends']
    mandatory_available = [v for v in mandatory_candidates if v in data.columns]
    
    print(f"\n🔒 Mandatory variables: {len(mandatory_available)}")
    for var in mandatory_available:
        print(f"   • {var}")
    
    all_vars = all_impression_cols + mandatory_available
    
    # Clean data
    data = data.dropna(subset=[TARGET_VARIABLE] + all_vars)
    print(f"\n✓ {len(data)} rows after cleaning")
    
    # Show constraints
    y = data[TARGET_VARIABLE].values
    print(f"\n📊 PREDICTION CONSTRAINTS:")
    print(f"   Min allowed:     {MIN_PREDICTION}")
    print(f"   Max allowed:     {np.max(y) * MAX_PREDICTION_MULTIPLIER:.0f} (3x historical max)")
    print(f"   Historical range: [{np.min(y):.0f}, {np.max(y):.0f}]")
    print(f"   Log transform:   {USE_LOG_TRANSFORM}")
    
    # Forward selection with constraints
    selected_vars, _ = forward_selection(
        data, all_vars, TARGET_VARIABLE,
        max_vars=MAX_VARIABLES,
        mandatory_vars=MANDATORY_VARIABLES,
        q=Q_VALUE, r=R_VALUE, init_cov=INIT_COV,
        use_log=USE_LOG_TRANSFORM
    )
    
    # Get dates
    dates = pd.to_datetime(data['Date']) if 'Date' in data.columns else np.arange(len(data))
    
    # Standardize
    if STANDARDIZE:
        scaler = StandardScaler()
        data[selected_vars] = scaler.fit_transform(data[selected_vars])
    
    X = data[selected_vars].values
    y = data[TARGET_VARIABLE].values
    
    # Tune with constraints
    if AUTO_TUNE_HYPERPARAMETERS:
        best_params = hyperparameter_tuning(X, y, X.shape[1], use_log=USE_LOG_TRANSFORM)
        q_opt, r_opt, init_cov_opt = best_params
    else:
        q_opt, r_opt, init_cov_opt = Q_VALUE, R_VALUE, INIT_COV
    
    # Final model with constraints
    print(f"\n⚙️ Running constrained Kalman Filter...")
    kf = ConstrainedTVLinearKalman(
        n_features=X.shape[1], q=q_opt, r=r_opt, init_cov=init_cov_opt,
        min_pred=MIN_PREDICTION, 
        max_pred=np.max(y) * MAX_PREDICTION_MULTIPLIER,
        use_log=USE_LOG_TRANSFORM
    )
    betas, y_pred = kf.fit(X, y)
    
    # Check constraints
    print(f"\n🔍 PREDICTION CHECK:")
    print(f"   Predicted range: [{np.min(y_pred):.0f}, {np.max(y_pred):.0f}]")
    print(f"   Negative predictions: {(y_pred < 0).sum()} ({'✓ NONE!' if (y_pred < 0).sum() == 0 else '✗ FOUND'})")
    print(f"   Extreme predictions: {(y_pred > np.max(y)*3).sum()} (>3x max)")
    
    # Metrics
    metrics = calculate_metrics(y, y_pred)
    print(f"\n📊 RESULTS:")
    print(f"{'─'*80}")
    print(f"  MAE:   {metrics['mae']:>10.2f}")
    print(f"  RMSE:  {metrics['rmse']:>10.2f}")
    print(f"  MAPE:  {metrics['mape']:>10.2f}%")
    print(f"  R²:    {metrics['r2']:>10.4f}")
    
    # Coefficients
    print(f"\n📋 FINAL COEFFICIENTS:")
    for i, var in enumerate(selected_vars):
        print(f"  {var[:50]:.<50} {betas[-1, i]:>10.4f}")
    
    # Plots
    print(f"\n📊 Generating plots...")
    plot_results(y, y_pred, dates, betas, selected_vars, TARGET_VARIABLE)
    
    print(f"\n{'='*80}")
    print(f"✅ COMPLETE - NO NEGATIVE OR EXTREME PREDICTIONS!")
    print(f"{'='*80}\n")


if __name__ == "__main__":
    run_analysis()

🔬 KALMAN FILTER - CONSTRAINED VERSION

📁 Loading data...
✓ 60 rows for 'interlocking diamond necklace'

🔒 Mandatory variables: 3
   • Product variant price
   • Category Discount
   • google_trends

✓ 60 rows after cleaning

📊 PREDICTION CONSTRAINTS:
   Min allowed:     0
   Max allowed:     36 (3x historical max)
   Historical range: [0, 12]
   Log transform:   False

🔍 FORWARD STEPWISE SELECTION (WITH CONSTRAINTS)

🔒 Mandatory variables (always included):
   1. Product variant price

📊 Will select 2 more variables from 26 candidates...

📍 Step 1: Testing 26 variables...
   ✓ Added: interlocking diamond necklace_meta_impression
     R² = -0.0164, MAE = 1.95

📍 Step 2: Testing 25 variables...
   ✓ Added: Impressions
     R² = 0.0356, MAE = 1.88

📊 SELECTION SUMMARY
 Step                                      Variable        R2      MAE  Num_Vars
    1 interlocking diamond necklace_meta_impression -0.016356 1.945821         2
    2                                   Impressions  0.035594 


🔧 Hyperparameter tuning...
   Best: Q=0.001, R=0.1, R²=0.1295

⚙️ Running constrained Kalman Filter...

🔍 PREDICTION CHECK:
   Predicted range: [0, 9]
   Negative predictions: 0 (✓ NONE!)
   Extreme predictions: 0 (>3x max)

📊 RESULTS:
────────────────────────────────────────────────────────────────────────────────
  MAE:         1.84
  RMSE:        2.61
  MAPE:       61.45%
  R²:        0.1295

📋 FINAL COEFFICIENTS:
  Product variant price.............................     1.0116
  interlocking diamond necklace_meta_impression.....     2.0407
  Impressions.......................................     0.0514

📊 Generating plots...



✅ COMPLETE - NO NEGATIVE OR EXTREME PREDICTIONS!

